In [121]:
import pandas as pd
import numpy as np
import xgboost as xgb
from sklearn.model_selection import StratifiedKFold, RandomizedSearchCV
from sklearn.metrics import mean_squared_error
from scipy.stats import randint as sp_randint, uniform as sp_uniform
import xgboost as xgb

In [122]:
# Read the final dataset from Parquet
train  = pd.read_csv("../final_dataset_feature_engineering.csv", engine="pyarrow", index_col='card_id')
train 

,authorized_flag,city_id_x,category_1_x,installments,category_3,merchant_category_id_x,merchant_id,month_lag,purchase_amount,purchase_date,...,city_id_y,state_id_y,category_2_y,feature_1,feature_2,feature_3,target,feature_1_2,feature_2_3,installments_log
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_7cfb53771e,0,-1,1,1,1,755,5999,-6,-0.716855,11113,...,69.0,9.0,1.0,2.0,1.0,0.0,2.384050,2.0,0.0,0.693147
C_ID_8f6e73a2a9,1,344,0,0,0,387,10961,-8,-0.731881,11909,...,69.0,9.0,1.0,3.0,2.0,1.0,-1.131020,6.0,2.0,0.000000
C_ID_f29a005fba,1,-1,1,1,1,839,20064,-1,-0.722745,39391,...,69.0,9.0,1.0,1.0,1.0,0.0,0.499893,1.0,0.0,0.693147
C_ID_9b1f626be5,0,69,0,1,1,792,9022,-6,-0.296112,17292,...,69.0,9.0,1.0,2.0,1.0,0.0,1.148530,2.0,0.0,0.693147
C_ID_c32ea06010,1,183,0,1,1,307,19574,-4,-0.701828,22794,...,69.0,9.0,1.0,4.0,1.0,0.0,-0.400508,4.0,0.0,0.693147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C_ID_c0b7f64338,1,53,0,-1,-1,434,9264,1,1.056274,44615,...,69.0,9.0,1.0,3.0,3.0,1.0,-2.151255,9.0,3.0,0.000000
C_ID_3879008bb7,1,69,0,0,0,606,17943,2,-0.737787,46394,...,69.0,9.0,1.0,4.0,2.0,0.0,0.569515,8.0,0.0,0.000000
C_ID_6614243228,1,-1,1,1,1,834,2440,2,-0.328840,45960,...,69.0,9.0,1.0,3.0,1.0,1.0,1.700309,3.0,1.0,0.693147


In [123]:
test = pd.read_csv("../test_feature_engineering.csv", engine="pyarrow", index_col='card_id')
test

,authorized_flag,city_id_x,category_1_x,installments,category_3,merchant_category_id_x,merchant_id,month_lag,purchase_amount,purchase_date,...,category_4,city_id_y,state_id_y,category_2_y,feature_1,feature_2,feature_3,feature_1_2,feature_2_3,installments_log
card_id,,,,,,,,,,,,,,,,,,,,,
C_ID_a8c9d4d2b1,1,-1,1,4,2,879,8697,-9,0.293783,5857,...,-1,82.0,9.0,1.0,2,1,0,2,0,1.609438
C_ID_5cc67e30b7,1,19,0,1,1,68,10298,0,-0.717005,1854,...,-1,82.0,9.0,1.0,4,2,0,8,0,0.693147
C_ID_5cc67e30b7,1,19,0,1,1,683,7458,0,-0.705405,1652,...,-1,82.0,9.0,1.0,4,2,0,8,0,0.693147
C_ID_5cc67e30b7,1,19,0,1,1,108,1689,0,-0.701828,1398,...,-1,82.0,9.0,1.0,4,2,0,8,0,0.693147
C_ID_9aa9b08732,1,344,0,-1,-1,34,14463,-2,-0.619182,20933,...,-1,82.0,9.0,1.0,2,1,0,2,0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
C_ID_5897de3c5c,1,146,0,1,1,705,10752,-2,-0.460668,14025,...,-1,82.0,9.0,1.0,4,3,0,12,0,0.693147
C_ID_5897de3c5c,1,146,0,1,1,506,10606,-3,-0.651520,11850,...,-1,82.0,9.0,1.0,4,3,0,12,0,0.693147
C_ID_5897de3c5c,1,-1,1,6,2,690,12877,-1,1.238425,15985,...,-1,82.0,9.0,1.0,4,3,0,12,0,1.945910


In [124]:
train['outliers'] = 0
train[train['target'] < -30]['outliers'] = 1

C:\Users\BISRI_MUSTAFA\AppData\Local\Temp\ipykernel_15196\3083907656.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train[train['target'] < -30]['outliers'] = 1


In [125]:
print('The shape of featured train data :',train.shape)
print('The shape of featured test data :',test.shape)

The shape of featured train data : (48095, 42)
The shape of featured test data : (29706, 40)


In [126]:
# splitting train and target values
y_train = train['target']

X = train.drop(['target'],axis=1)

outliers = X['outliers']

X_train = X.drop(['outliers'],axis = 1)

In [127]:
import xgboost as xgb
from scipy.stats import randint as sp_randint
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import RandomizedSearchCV, KFold

# Inisialisasi model
xgb_model = xgb.XGBRegressor(
    learning_rate=0.01,
    n_estimators=100,
    objective='reg:squarederror',
    eval_metric='rmse',
    verbosity=0,
    nthread=8,
    tree_method='hist'  # Ganti dari 'gpu_hist' ke 'hist'
)

# Parameter untuk hyperparameter tuning
parameters = {
    'max_depth': [3, 4, 5, 6],
    'subsample': [i / 10.0 for i in range(6, 10)],
    'colsample_bytree': [i / 10.0 for i in range(6, 10)],
    'min_child_weight': sp_randint(1, 11),
    'gamma': [i / 10.0 for i in range(0, 5)],
    'reg_alpha': [1e-5, 1e-2, 0.1, 1, 100]
}

# KFold cross-validation
cv = KFold(n_splits=9, shuffle=True, random_state=1989)

# Randomized search
random_search = RandomizedSearchCV(
    estimator=xgb_model,
    param_distributions=parameters,
    n_iter=5,
    scoring='neg_mean_squared_error',
    cv=cv,
    verbose=3,
    n_jobs=-1,
    error_score='raise'
)

# Fit model
random_search.fit(X_train, y_train)


Fitting 9 folds for each of 5 candidates, totalling 45 fits


RandomizedSearchCV(cv=KFold(n_splits=9, random_state=1989, shuffle=True),
                   error_score='raise',
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=False,
                                          eval_metric='rmse',
                                          feature_types=None,
                                          feature_weights=None, ga...
                                          n_jobs=None, nthread=8, ...),
                   n_iter=5, n_jobs=-1,
                   param_distributions={'colsample_bytree': [0.6, 0.7, 0.8,
                                                             0.9],
                                        'gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
                                        'max_depth': [3, 4, 5, 6],
                                        'min_child_weight': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x000002A9E257C5D0>,
                                        'reg_alpha': [1e-05, 0.01, 0.1, 1, 100],
                                        'subsample': [0.6, 0.7, 0.8, 0.9]},
                   scoring='neg_mean_squared_error', verbose=3)

In [128]:
#printing the best parameters got from the RandomizedSearchCV.
random_search.best_params_

{'colsample_bytree': 0.9,
 'gamma': 0.0,
 'max_depth': 5,
 'min_child_weight': 3,
 'reg_alpha': 1,
 'subsample': 0.8}

In [129]:
xgb_params = {
    'eta': 0.1,
    'max_depth': 6,
    'subsample': 0.9,
    'colsample_bytree': 0.9,
    'learning_rate': 0.01,
    'gamma': 0.1,
    'min_child_weight': 3,
    'reg_alpha': 1e-05,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse',
    'verbosity': 0,  # Changed from 'silent'
    'tree_method': 'hist'
}

predictions_xgb_train = np.zeros(len(X_train))
predictions_xgb_test = np.zeros(len(test))

# Cross-validation setup
FOLDs = KFold(n_splits=9, shuffle=True, random_state=1989)

for fold_, (trn_idx, val_idx) in enumerate(FOLDs.split(X_train)):
    print(f"xgb {fold_} " + "-" * 50)
    
    # Prepare data
    trn_data = xgb.DMatrix(X_train.iloc[trn_idx], label=y_train.iloc[trn_idx])
    val_data = xgb.DMatrix(X_train.iloc[val_idx], label=y_train.iloc[val_idx])
    
    # Train model
    xgb_model = xgb.train(
        xgb_params,
        trn_data,
        num_boost_round=1000,
        evals=[(trn_data, 'train'), (val_data, 'valid')],
        early_stopping_rounds=200,
        verbose_eval=200
    )
    
    # Get best iteration
    best_iter = xgb_model.best_iteration
    
    # Predict validation set - using only up to best_iter
    predictions_xgb_train[val_idx] = xgb_model.predict(
        xgb.DMatrix(X_train.iloc[val_idx]),
        iteration_range=(0, best_iter)  # Removed +50 to prevent overflow
    )
    
    # Predict test set and average across folds
    predictions_xgb_test += xgb_model.predict(
        xgb.DMatrix(test),
        iteration_range=(0, best_iter)
    ) / FOLDs.n_splits

xgb 0 --------------------------------------------------
[0]	train-rmse:3.57341	valid-rmse:3.67509
[200]	train-rmse:3.27164	valid-rmse:3.48207
[400]	train-rmse:3.15641	valid-rmse:3.41638
[600]	train-rmse:3.07442	valid-rmse:3.38606
[800]	train-rmse:3.00265	valid-rmse:3.36487
[999]	train-rmse:2.94174	valid-rmse:3.34738
xgb 1 --------------------------------------------------
[0]	train-rmse:3.58874	valid-rmse:3.55338
[200]	train-rmse:3.27474	valid-rmse:3.38708
[400]	train-rmse:3.15444	valid-rmse:3.33667
[600]	train-rmse:3.07497	valid-rmse:3.31366
[800]	train-rmse:3.00208	valid-rmse:3.29082
[999]	train-rmse:2.93713	valid-rmse:3.27138
xgb 2 --------------------------------------------------
[0]	train-rmse:3.60825	valid-rmse:3.38788
[200]	train-rmse:3.29602	valid-rmse:3.21511
[400]	train-rmse:3.18155	valid-rmse:3.15275
[600]	train-rmse:3.09319	valid-rmse:3.11559
[800]	train-rmse:3.02252	valid-rmse:3.09311
[999]	train-rmse:2.96057	valid-rmse:3.07586
xgb 3 -------------------------------------

In [130]:
#RMSE score for Xgboost model
print(np.sqrt(mean_squared_error(predictions_xgb_train, y_train)))

3.2869631265414077


In [131]:
# saving the predcitions 
np.save('predictions_xgb_train.npy', predictions_xgb_train)
np.save('predictions_xgb_test.npy', predictions_xgb_test)

In [132]:

import pickle
# saving the model 
pickle.dump(xgb_model, open('../xgb_model_final.sav', 'wb'))

In [138]:
submit_df = pd.read_csv('../final_test_feature_engineering.csv')
submit_df["target_score"] = predictions_xgb_test

# Ambil kolom dan urutkan secara descending berdasarkan target_score
submit_df = submit_df[["card_id", "target_score"]].sort_values(by="target_score", ascending=False)

submit_df.to_csv("hasil_test.csv", index=False)

submit_df.head(10)


,card_id,target_score
1933,C_ID_c86d50f05d,1.180516
5595,C_ID_fcbd58bbaa,0.720080
25888,C_ID_5d5b020320,0.682485
2439,C_ID_11ea062254,0.674104
7911,C_ID_d3c2db358c,0.639106
2084,C_ID_bd61169c83,0.634303
22806,C_ID_9831e32564,0.624514
23010,C_ID_f33fe8a892,0.602465
16399,C_ID_1f9f776021,0.602275
2349,C_ID_7c06b655a4,0.601418
